In [44]:
import gdown
import zipfile
import os
import faiss
import datasets

import typing
import time
from typing import Tuple, Dict
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import load_dataset
import json
import numpy as np

from transformers import AutoTokenizer, RagRetriever, RagSequenceForGeneration, RagConfig, AutoConfig, AutoModel, \
    RagTokenizer, BartForConditionalGeneration, AlbertModel, Trainer, TrainingArguments, BatchEncoding

import torch
from torch.utils.data import DataLoader

from transformers.modeling_outputs import BaseModelOutputWithPooling

from reqs.lightning_base import BaseTransformer
from reqs.utils import *

## Loading the Dataset

In [12]:
url = "https://drive.google.com/uc?id=18xMA2wGPDXArwLyVWN3HXQaF0XnjtugF"
filepath = "data/gold"

# Check if index exists
if os.path.isfile(filepath + "/index.faiss"):
    print("File already exists")
else:

    # Download zip file using gdown
    gdown.download(url, "index.zip", quiet=False)

    # Create directory if it doesn't exist
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    # Unzip file
    with zipfile.ZipFile("index.zip", 'r') as zip_ref:
        zip_ref.extractall(filepath)

    # Remove zip file
    os.remove("index.zip")

File already exists


## Creating the Model

In [13]:
encoder_model_name = "sentence-transformers/paraphrase-albert-base-v2"
encoder_model_type = "albert"
encoder_config = AutoConfig.from_pretrained(encoder_model_name, output_hidden_states=True)

generator_model_name = "facebook/bart-base"
generator_model_type = "bart"
generator_config = AutoConfig.from_pretrained(generator_model_name)

In [14]:
rag_config = RagConfig(
    question_encoder={
        "model_type": encoder_model_type,
        "config": encoder_config,
    },
    generator = {
        "model_type": generator_model_type,
        "config": generator_config
    },
    index_name="custom",
    passages_path=filepath + "/dataset",
    index_path=filepath + "/index.faiss",
)

In [26]:

rag_retriever = RagRetriever(
    config=rag_config,
    question_encoder_tokenizer = AutoTokenizer.from_pretrained(encoder_model_name),
    generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name),
)

In [16]:
class CustomQuestionEncoder(AlbertModel):
    def forward(self, *args, **kwargs):
        # Call the original forward method
        outputs = super().forward(*args, **kwargs)
        attention_mask = kwargs.get('attention_mask', None)

        if attention_mask is None:
            # Assume all 1s if not given, use output to get mask. The final output must be two-dimensional
            attention_mask = torch.ones(outputs[0].shape[:2], device=outputs[0].device)


        token_embeddings = outputs[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        pooler_output = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        # Return pooler output, hidden states and attentions
        return BaseModelOutputWithPooling(pooler_output=pooler_output, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

# Use the custom question encoder
question_encoder_model = CustomQuestionEncoder.from_pretrained(encoder_model_name)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

rag_model = RagSequenceForGeneration(
    config=rag_config,
    retriever=rag_retriever,
    question_encoder=question_encoder_model,
    generator=BartForConditionalGeneration.from_pretrained(generator_model_name),
)

rag_tokenizer = RagTokenizer(
    question_encoder=AutoTokenizer.from_pretrained(encoder_model_name),
    generator=AutoTokenizer.from_pretrained(generator_model_name),
)

## Create dataset

In [18]:
# Create pytorch dataset
class QuestionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

        # Get max length from tokenizer
        self.max_length = 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data.iloc[idx]['question']
        question_encoding = self.tokenizer.question_encoder(question, return_tensors="pt")

        return {**question_encoding}

In [19]:


# Example: Load a dataset
questions_dataset = load_dataset("web_questions")

# Accessing data
print(questions_dataset["train"][0])
questions_dataset


# Use pd.json_normalize to convert the JSON to a DataFrame
questions_df = pd.json_normalize(questions_dataset["train"], meta=['url','question', 'answers'])

# Split the dataset into training and validation
train_df, val_df = train_test_split(questions_df, test_size=0.3)

Generating train split: 0 examples [00:00, ? examples/s]

{'qText': 'what is the name of justin bieber brother?', 'qId': 'wqr000000', 'answers': ['Jazmyn Bieber', 'Jaxon Bieber']}


In [20]:
tokenizer = AutoTokenizer.from_pretrained(encoder_model_name)
train_dataset = QuestionDataset(train_df, rag_tokenizer)
val_dataset = QuestionDataset(val_df, rag_tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="trainer",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
)

# Instantiate the Trainer
trainer = Trainer(
    model=question_encoder_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)

trainer.train()

## Initial Testing

In [23]:
rag_model.to(device)

question = "What is the capital of the Netherlands"
inputs = rag_tokenizer.question_encoder(question, return_tensors="pt").to(device)

generated = rag_model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=50, num_beams=4, early_stopping=False)
generated_string = rag_tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

print("Question:", question)
print("Answer:", generated_string)

Question: What is the capital of the Netherlands
Answer: Netherlands / The Netherlands, informally Holland, is a country located in northwestern Europe with overseas territories in the Caribbean. It is the largest of the four constituent countries of the Kingdom of the Netherlands. The Netherlands consists of twelve provinces;


## Fine-tuning Code

In [40]:
# Adapted from https://github.com/huggingface/transformers/blob/main/examples/research_projects/rag/finetune_rag.py#L97

class GenerativeQAModule(BaseTransformer):
    mode = "generative_qa"
    loss_names = ["loss"]
    metric_names = ["em"] # exact match
    val_metric = "em"

    def __init__(self, hparams, **kwargs):
        # Global variables used here from simplicity

        self.retriever = rag_retriever
        prefix = rag_config.question_encoder.prefix

        super().__init__(hparams, config=rag_config, tokenizer=rag_tokenizer, model=rag_model)

        self.dataset_kwargs = {
            "prefix": prefix,
            "max_source_length": self.max_source_length,
            "data_dir": self.data_dir,
        }

    def forward(self, input_ids, **kwargs):
        return self.model(input_ids, **kwargs)
    
    def ids_to_clean_text(self, generated_ids):
        preds = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        return lmap(str.strip, preds)
    
    def _step(self, batch: dict) -> Tuple:
        source_ids, source_mask, target_ids = batch["input_ids"], batch["attention_mask"], batch["decoder_input_ids"]

        rag_kwargs = {}
        decoder_input_ids = target_ids
        lm_labels = decoder_input_ids
        rag_kwargs["reduce_loss"] = True

        assert decoder_input_ids is not None

        outputs = self(
            source_ids,
            attention_mask=source_mask,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            labels=lm_labels,
            **rag_kwargs,
        )

        loss = outputs["loss"]
        return (loss,)
    
    def training_step(self, batch, batch_idx) -> Dict:
        loss_tensors = self._step(batch)

        logs = {name: loss.detach() for name, loss in zip(self.loss_names, loss_tensors)}

        logs["tpb"] = (
            batch["input_ids"].ne(self.tokenizer.question_encoder.pad_token_id).sum() + 
            batch["decoder_input_ids"].ne(self.tokenizer.generator.pad_token_id).sum()
        )

        return {"loss": loss_tensors[0], "log": logs}
    
    def validation_step(self, batch, batch_idx) -> Dict:
        return self._generative_step(batch)
    
    def validation_epoch_end(self, outputs, prefix="val") -> Dict:
        self.step_count += 1
        losses = {k: torch.stack([x[k] for x in outputs]).mean() for k in self.loss_names}
        loss = losses["loss"]

        gen_metrics = {
            k: np.array([x[k] for x in outputs]).mean() for k in self.metric_names + ["gen_time", "gen_len"]
        }

        metrics_tensor: torch.FloatTensor = torch.tensor(gen_metrics[self.val_metric]).type_as(loss)
        gen_metrics.update({k: v.item() for k, v in losses.items()})

        losses.update(gen_metrics)
        metrics = {f"{prefix}_avg_{k}": x for k, x in losses.items()}
        preds = flatten_list([x["preds"] for x in outputs])
        
        return {"log": metrics, "preds": preds, f"{prefix}_loss": loss, f"{prefix}_{self.val_metric}": metrics_tensor}
    
    def calc_generative_metrics(self, preds, target) -> Dict:
        return calculate_exact_match(preds, target)
    
    def _generative_step(self, batch: dict) -> dict:
        
        start_time = time.time()
        batch = BatchEncoding(batch).to(device=self.model.device)
        generated_ids = self.model.generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            do_deduplication=False,  # rag specific parameter
            use_cache=True,
            min_length=1,
            max_length=self.target_lens["val"],
        )

        gen_time = (time.time() - start_time) / batch["input_ids"].shape[0]
        preds: List[str] = self.ids_to_clean_text(generated_ids)
        target: List[str] = self.ids_to_clean_text(batch["decoder_input_ids"])
        loss_tensors = self._step(batch)
        base_metrics = dict(zip(self.loss_names, loss_tensors))
        gen_metrics: Dict = self.calc_generative_metrics(preds, target)

        summ_len = np.mean(lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target, **gen_metrics)
        return base_metrics
    
    def test_step(self, batch, batch_idx):
        return self._generative_step(batch)

    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, prefix="test")
    
    def get_dataset(self, type_path) -> Seq2SeqDataset:
        n_obs = self.n_obs[type_path]
        max_target_length = self.target_lens[type_path]
        dataset = Seq2SeqDataset(
            self.tokenizer,
            type_path=type_path,
            n_obs=n_obs,
            max_target_length=max_target_length,
            **self.dataset_kwargs,
        )
        return dataset

    def get_dataloader(self, type_path: str, batch_size: int, shuffle: bool = False) -> DataLoader:
        dataset = self.get_dataset(type_path)

        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            collate_fn=dataset.collate_fn,
            shuffle=shuffle,
            num_workers=self.num_workers,
        )
        return dataloader

    def train_dataloader(self) -> DataLoader:
        dataloader = self.get_dataloader("train", batch_size=self.hparams.train_batch_size, shuffle=True)
        return dataloader

    def val_dataloader(self) -> DataLoader:
        return self.get_dataloader("val", batch_size=self.hparams.eval_batch_size)

    def test_dataloader(self) -> DataLoader:
        return self.get_dataloader("test", batch_size=self.hparams.eval_batch_size)
    

In [39]:
hparams = parse_sh_args("fine_tune_rag.sh")
QAModule = GenerativeQAModule(hparams)